This is the shortened version of the work in class2.ipynb - for more notes on functions and piple check that document or exercises 1 nad 2 from class 2 of the NLP course https://minaalmasi.github.io/nlp-at-cogsci/main/class2_intro.html 

In [1]:
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [6]:
%pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [12]:
from pathlib import Path
import pandas as pd

#Import BOW vectorizer for preprocessing

from sklearn.feature_extraction.text import CountVectorizer

#Import TF-IDF vectorizer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.metrics import classification_report

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

In [3]:
# path of notebook
path = Path.cwd()

data_path = path.parents[1] / "resources" / "data" / "raid" / "train_none.csv"

raw_df = pd.read_csv(data_path)

In [ ]:
#Pipeline without function
df = raw_df[raw_df["model"].isin(["human", "cohere"])]

df["is_human"] = df["model"].apply(lambda x: 1 if x == "human" else 0)

train_df, val_df = train_test_split(df, test_size=0.2, random_state=42, stratify= df['is_human'])


In [18]:
def datasplitter(raw_df, model_a, model_b, positive_class,
                 test_size, random_state):
    """
    Subset dataframe by two chosen models, create binary column based on
    positive_class, and split into train and validation sets.
    
    Parameters:
    -----------
    raw_df : pandas.DataFrame
        The original dataframe containing at least a 'model' column.
    model_a, model_b : str
        The two models to include in the subset.
    positive_class : str
        The model label that should be encoded as 1 in the binary column.
    test_size : float
        Proportion of validation set (default 0.2).
    random_state : int
        Random seed for reproducibility.
    
    Returns:
    --------
    train_df : pandas.DataFrame
    val_df   : pandas.DataFrame
    """
    
    # Subset
    df = raw_df[raw_df["model"].isin([model_a, model_b])].copy()
    
    # Binary column
    binary_col = f"is_{positive_class.lower()}"
    df[binary_col] = df["model"].apply(lambda x: 1 if x == positive_class else 0)
    
    # Split
    train_df, val_df = train_test_split(
        df,
        test_size=test_size,
        random_state=random_state,
        stratify=df[binary_col]
    )
    
    return train_df, val_df

In [19]:
train_df, val_df = datasplitter(raw_df, "human", "chatgpt", "human", 0.2, 52)

In [10]:
from typing import Literal 
# using Literal is not strictly necessary, 
# but is a way to define the options that you can use for the function!

def vectorize(X_train: pd.Series, X_val: pd.Series, vec_type:Literal["bow", "tf-idf"], max_features:int=500):
    """
    Function to vectorize train and val data! 
    """
    if vec_type == "bow":
        vectorizer = CountVectorizer(lowercase=True, max_features=max_features)
    elif vec_type== "tf-idf":
        vectorizer = TfidfVectorizer(lowercase=True, max_features=max_features)
    else: 
        # this is good code practice, but if your function has no 'else' statement, this is also fine!
        raise ValueError(f"Invalid vec_type: {vec_type}. Must be either 'bow' or 'tf-idf")
    
    X_train_vectorized = vectorizer.fit_transform(X_train)
    X_val_vectorized = vectorizer.transform(X_val)

    return X_train_vectorized, X_val_vectorized

In [11]:
# APPLY TO GET TF-IDF
X_train_tfidf, X_val_tfidf = vectorize(
                        X_train = train_df["generation"], 
                          X_val = val_df["generation"],
                          vec_type="tf-idf",
                          max_features=500
                          )

In [ ]:
# clf is common naming convention
clf = LogisticRegression(
    random_state=42,
    solver="liblinear",   # tested all solvers, and this one was fastest ... (should also be appropriate for our data)
    max_iter=1000,        # allow more iterations
    C=1.0,                # adjust if needed (smaller values = stronger regularization)
)

#Let’s extract our numerical labels as Y:

y_train = train_df["is_human"]

#Let’s fit our classifier on our vectorized text X_train_bow and our y_train:

clf.fit(X_train_tfidf, y_train)

#Let’s extract predicted labels y:

y_pred_tfidf = clf.predict(X_val_tfidf)

#Now, we can compare to our actual labels y val:

y_val_tfidf = val_df["is_human"]
report_tfidf = classification_report(y_val_tfidf, y_pred_tfidf)
print(report_tfidf)

In [ ]:
#This function needs some work: I want to just pass a data frame and then it can extract y values itself

def classifier(X_train, y_train, X_val, y_val, model_name,
                               random_state, solver,
                               max_iter, C, verbose=True):
    """
    Train logistic regression, predict on validation data, 
    and print classification report.
    
    Parameters:
    -----------
    X_train, y_train : training data + labels
    X_val, y_val     : validation data + labels
    random_state     : int, seed
    solver           : str, optimizer
    max_iter         : int, max iterations
    C                : float, inverse regularization strength
    verbose          : bool, whether to print report
    
    Returns:
    --------
    clf   : trained LogisticRegression model
    y_pred: predicted labels for validation set
    report: classification report (string)
    """
    
    # Train
    model_name = LogisticRegression(
        random_state=random_state,
        solver=solver,
        max_iter=max_iter,
        C=C
    )
    model_name.fit(X_train, y_train)
    
    # Predict
    y_pred = model_name.predict(X_val)
    
    # Report
    report = classification_report(y_val, y_pred)
    if verbose:
        print(report)
    
    return model_name, y_pred, report

In [24]:
y_train = train_df["is_human"]

y_val_tfidf = val_df["is_human"]

clf, y_pred, report = classifier(X_train_tfidf, y_train, X_val_tfidf, y_val_tfidf, clf, 52, "liblinear", 1000, 1.0 )

              precision    recall  f1-score   support

           0       0.67      1.00      0.80      5349
           1       0.14      0.00      0.00      2674

    accuracy                           0.67      8023
   macro avg       0.40      0.50      0.40      8023
weighted avg       0.49      0.67      0.53      8023

